In [2]:
pip install delta-spark==3.3.2 dotenv

  Using cached pyspark-3.5.7-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.0
    Can't uninstall 'pyspark'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from datetime import datetime

In [4]:
from dotenv import load_dotenv
import os
load_dotenv('/opt/workspace/.env')
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT_DOCKER")
MINIO_ACCESS = os.getenv("MINIO_ROOT_USER")
MINIO_SECRET = os.getenv("MINIO_ROOT_PASSWORD")
                           
today = datetime.now().strftime("%Y/%m/%d")
SILVER_PATH = f"s3a://silver/posicao/"
GOLD_PATH = "s3a://gold/posicao/"
print(f"Buscando delta em: {SILVER_PATH}")

Buscando delta em: s3a://silver/posicao/


In [5]:
builder = (
    SparkSession.builder.appName("SilverToGold_Delta")
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # Delta Lake
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [17]:
today = "data_ref=" + datetime.today().strftime('%Y-%m-%d')
print(today)
SILVER_LINHAS_PATH = f"s3a://silver/linhas/{today}"
SILVER_ROUTES_PATH = "s3a://silver/gtfs/routes/"
GOLD_DIM_LINHA_PATH = "s3a://gold/dim_linha/"

print(f"📂 Lendo Silver Linhas: {SILVER_LINHAS_PATH}")
print(f"📂 Lendo Silver GTFS Routes: {SILVER_ROUTES_PATH}")

data_ref=2025-11-10
📂 Lendo Silver Linhas: s3a://silver/linhas/data_ref=2025-11-10
📂 Lendo Silver GTFS Routes: s3a://silver/gtfs/routes/


In [21]:
# Leitura
df_api = spark.read.format("delta").load(SILVER_LINHAS_PATH)
df_gtfs = spark.read.format("delta").load(SILVER_ROUTES_PATH)
df_api = df_api.alias("api")
df_gtfs = df_gtfs.alias("gtfs")

In [22]:
# Join por letreiro/codigo_linha
dim_linha = (
    df_api
    .join(
        df_gtfs,
        F.lower(F.col("api.letreiro")) == F.lower(F.col("gtfs.letreiro")),
        "left"
    )
    .select(
        F.col("api.codigo_linha"),
        F.col("api.letreiro"),
        F.col("api.tipo_linha"),
        F.col("api.linha_circular"),
        F.col("api.terminal_principal"),
        F.col("api.terminal_secundario"),
        F.col("gtfs.route_color"),
        F.col("gtfs.route_text_color"),
        F.current_date().alias("data_ref"),
        F.current_timestamp().alias("ingest_timestamp"),
    )
)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `api`.`tipo_linha` cannot be resolved. Did you mean one of the following? [`api`.`codigo_linha`, `api`.`sentido_linha`, `api`.`letreiro`, `api`.`data_ref`, `api`.`modo_circular`].;
'Project [codigo_linha#6736L, letreiro#6738, 'api.tipo_linha, 'api.linha_circular, terminal_principal#6740, terminal_secundario#6741, route_color#6754, route_text_color#6755, current_date(Some(Etc/UTC)) AS data_ref#6794, current_timestamp() AS ingest_timestamp#6795]
+- Join LeftOuter, (lower(letreiro#6738) = lower(letreiro#6753))
   :- SubqueryAlias api
   :  +- Filter (data_ref#6742 = cast(2025-11-10 as date))
   :     +- Relation [codigo_linha#6736L,modo_circular#6737,letreiro#6738,sentido_linha#6739L,terminal_principal#6740,terminal_secundario#6741,data_ref#6742,ingest_timestamp#6743] parquet
   +- SubqueryAlias gtfs
      +- Relation [letreiro#6753,route_color#6754,route_text_color#6755,nome_publico#6756,termi